In [13]:
import os

In [22]:
%pwd

'/'

In [23]:
os.chdir("/Users/matt/Documents/deeplearning")
%pwd

'/Users/matt/Documents/deeplearning'

In [24]:
from dataclasses import dataclass
from pathlib import Path  
 
@dataclass(frozen=True) #表示这是不可变对象，初始化后不能重新赋值
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file:Path
    unzip_dir:Path    

In [25]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [37]:
class ConfigurationManager:
  def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])
    
  def get_data_ingestion_config(self)->DataIngestionConfig:
    config=self.config.data_ingestion
    
    create_directories([config.root_dir])
    
    data_ingestion_config=DataIngestionConfig(
      root_dir=config.root_dir,
      source_URL=config.source_URL,
      local_data_file=config.local_data_file,
      unzip_dir=config.unzip_dir
    )
    return data_ingestion_config

In [39]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [44]:
class DataIngestion:
  def __init__(self,config:DataIngestionConfig):
    self.config = config
    
  def download_file(self):
    try:
      if not os.path.exists(self.config.local_data_file):
        filename,headers=request.urlretrieve(
          url=self.config.source_URL,
          filename=self.config.local_data_file
          )
        logger.info(f"{filename} downloaded! with following information:\n {headers}")
      else:
        logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}")
    except Exception as e:
      logger.error(f"Error downloading {self.config.local_data_file}: {e}")
      # raise e
      
  def extract_zip_file(self):
    try:
      unzip_path = self.config.unzip_dir
      os.makedirs(unzip_path,exist_ok=True)
      with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
        zip_ref.extractall(path=unzip_path)
      logger.info(f"{self.config.local_data_file} extracted to {unzip_path}")
    except Exception as e:
      logger.error(f"Error extracting file {self.config.local_data_file} to {unzip_path}: {e}")
      # raise e

In [45]:
try:
  config_manager =ConfigurationManager()
  data_ingestion_config=config_manager.get_data_ingestion_config()
  data_ingestion=DataIngestion(data_ingestion_config)
  data_ingestion.download_file()
  data_ingestion.extract_zip_file()
except Exception as e:
  print(f"error: {e}")


[2023-10-15 14:52:52,889: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-15 14:52:52,900: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-15 14:52:52,916: INFO: common: created directory at: artifacts]


[2023-10-15 14:52:52,922: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-15 14:52:52,927: INFO: 3488843591: File already exists of size ~ 11345 KB]
[2023-10-15 14:52:54,800: INFO: 3488843591: artifacts/data_ingestion/data.zip extracted to artifacts/data_ingestion]


# deeplearning
This repository  is a deep learning repository. 

## workflow
1. Update config.yaml
2. Update secrets.yaml [Optional]
3. Update params.yaml
4. Update the entity
5. Update the configuration manager in src config
6. Update the components
7. Update the pipeline
8. Update the main.py
9. Update the dvc.yaml